In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv


In [46]:
df = pd.read_csv("/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv")

In [47]:
print(df.shape)
df.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [48]:
# checking missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [49]:
# checking total number of duplicated rows
df.duplicated().sum()

0

In [50]:
# Number of records of people enrolled to bank
df['Exited'].value_counts()
# As we can see the below values, there is imbalance in the classification

Exited
0    7963
1    2037
Name: count, dtype: int64

In [11]:
df['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [12]:
df['Gender'].value_counts()

Gender
Male      5457
Female    4543
Name: count, dtype: int64

In [51]:
# Dropping irrelevant columns
df.drop(columns = ["RowNumber", "CustomerId", "Surname"], inplace = True)
# inplace = True makes the changes permanant

In [52]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [54]:
# performing one-hot encoding for categorical data
import pandas as pd

# Assuming df is your DataFrame
df = pd.get_dummies(df, columns=["Geography", "Gender"], drop_first=True, dtype=np.int64)

# We are dropping the firt column as 3 categories can be represented by 2 column and 2 category of gender column in one column

In [ ]:
# Scaling all the columns as we can see that the values of the column is not comparable
# values of salary columns are much much higher
# We are scaling beacuse in the time of calculation of weights it takes very long to converge


In [55]:
# Splitting data
X = df.drop(columns = ['Exited'])
y = df['Exited']

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [57]:
print(X_train.shape)
print(X_test.shape)

(8000, 11)
(2000, 11)


In [58]:
# Scaling the training and test data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_trained_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

'''Scaling is a crucial step in many machine learning algorithms, and it involves standardizing or normalizing the features of your dataset. The reason for using different functions for scaling the training and testing data is to prevent data leakage and ensure that your model generalizes well to new, unseen data.

When you scale your data, you calculate statistics (like mean and standard deviation) based on the training set and use these statistics to scale both the training and testing sets. This is why you use fit_transform on the training data—it calculates the scaling parameters (mean and standard deviation) and applies the transformation to the training set.

However, when it comes to the testing set, you don't want to recalculate the scaling parameters because it's important that the scaling is consistent with what the model learned during training. If you used the statistics from the testing set to scale it, the model would essentially have information about the testing set during training, leading to optimistic and inaccurate performance estimates.

So, by using transform on the testing set, you apply the same scaling transformation that was learned from the training set. This ensures that the testing data is scaled in the same way as the training data, maintaining the integrity of the model evaluation on unseen data.'''


"Scaling is a crucial step in many machine learning algorithms, and it involves standardizing or normalizing the features of your dataset. The reason for using different functions for scaling the training and testing data is to prevent data leakage and ensure that your model generalizes well to new, unseen data.\n\nWhen you scale your data, you calculate statistics (like mean and standard deviation) based on the training set and use these statistics to scale both the training and testing sets. This is why you use fit_transform on the training data—it calculates the scaling parameters (mean and standard deviation) and applies the transformation to the training set.\n\nHowever, when it comes to the testing set, you don't want to recalculate the scaling parameters because it's important that the scaling is consistent with what the model learned during training. If you used the statistics from the testing set to scale it, the model would essentially have information about the testing set d

In [60]:
X_trained_scaled

array([[-0.23082038, -0.94449979, -0.70174202, ...,  1.71490137,
        -0.57273139,  0.91509065],
       [-0.25150912, -0.94449979, -0.35520275, ..., -0.58312392,
        -0.57273139, -1.09278791],
       [-0.3963303 ,  0.77498705,  0.33787579, ...,  1.71490137,
        -0.57273139, -1.09278791],
       ...,
       [ 0.22433188,  0.58393295,  1.3774936 , ..., -0.58312392,
        -0.57273139, -1.09278791],
       [ 0.13123255,  0.01077067,  1.03095433, ..., -0.58312392,
        -0.57273139, -1.09278791],
       [ 1.1656695 ,  0.29735181,  0.33787579, ...,  1.71490137,
        -0.57273139,  0.91509065]])

In [62]:
pip install tensorFlow

Note: you may need to restart the kernel to use updated packages.


In [66]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# There are two types of model 